In [65]:
import numpy as np

In [93]:
X_train, y_train, X_test, y_test, X_dev, y_dev = np.load('./all_data.npy', allow_pickle = True)

In [67]:
def make_X(x):
    res = []
    x = x[767:]
    for i in range(0, x.shape[0], 768):
#         print(i)
        res.append(np.array(x[i:i+768]))
    print(np.array(res).shape)
    return np.array(res)

In [68]:
X_train, X_test, X_dev = make_X(X_train), make_X(X_test), make_X(X_dev)

(9989, 768)
(2610, 768)
(1109, 768)


In [69]:
X_train[0]

array([-6.40730262e-01, -3.37642372e-01, -7.80585647e-01,  4.76331055e-01,
        4.17819947e-01, -1.38198420e-01,  5.06262422e-01,  1.64545968e-01,
       -3.86585325e-01, -9.99690473e-01,  2.87325550e-02,  7.08387673e-01,
        9.43858206e-01,  4.84381765e-01,  7.77700901e-01, -5.24473667e-01,
        1.65937319e-01, -5.27704179e-01,  2.51430839e-01,  1.38695523e-01,
        6.51594698e-01,  9.99777853e-01,  3.21779698e-01,  3.14313889e-01,
        3.95222604e-01,  7.45978653e-01, -3.44828039e-01,  8.21617901e-01,
        8.93230081e-01,  6.90777659e-01, -4.38903183e-01,  1.63439080e-01,
       -9.74046052e-01, -1.60484880e-01, -7.69122422e-01, -9.58322704e-01,
        2.45081514e-01, -5.41340768e-01,  3.96471610e-03,  1.29146457e-01,
       -7.96119213e-01,  3.03336799e-01,  9.99148011e-01, -5.66293418e-01,
        3.39183331e-01, -2.42660180e-01, -9.99906719e-01,  3.50715160e-01,
       -7.59753048e-01,  5.67711353e-01,  5.44875026e-01,  5.25920868e-01,
        1.47370964e-01,  

In [70]:
from sklearn import preprocessing

min_max_scaler = preprocessing.StandardScaler()
X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.transform(X_test)
X_dev = min_max_scaler.transform(X_dev)

In [71]:
num_classes = 7
time_step = 1

In [72]:
def expand_on_time(X, timestep = time_step):
    tmp = np.reshape(X[0], (1, len(X[0])))
#     print(tmp.shape)
    tmp = np.repeat(tmp, timestep, axis = 0)
#     print(tmp.shape)
    ab = np.reshape(tmp, (1, len(tmp[:,:]), len(tmp[0])))
#     print(ab.shape)
#     print(X.shape)
    X = np.repeat(ab, X.shape[0], axis = 0)
#     print(X.shape)
    return X

In [73]:
X_train, X_test, X_dev = expand_on_time(X_train), expand_on_time(X_test), expand_on_time(X_dev)

In [74]:
X_train.shape, X_test.shape, X_dev.shape

((9989, 1, 768), (2610, 1, 768), (1109, 1, 768))

In [75]:
import pandas as pd

loly = pd.DataFrame(y_train, columns = range(0,7))

In [76]:
loly[0].value_counts().loc[1]

1109

In [77]:
arr = [0]*7
for i in range(7):
    arr[i] = loly[i].value_counts().loc[1]
# arr = np.array(arr)
mx = max(arr)
for i in range(7):
    print(arr[i], mx, mx/arr[i])
    arr[i] = mx/arr[i]
# arr = 1.5*(1 - arr/max(arr))
arr

1109 4710 4.247069431920649
271 4710 17.380073800738007
268 4710 17.574626865671643
1743 4710 2.70223752151463
4710 4710 1.0
683 4710 6.8960468521229865
1205 4710 3.908713692946058


[4.247069431920649,
 17.380073800738007,
 17.574626865671643,
 2.70223752151463,
 1.0,
 6.8960468521229865,
 3.908713692946058]

In [78]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [79]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5324554589689648367
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 23526814592
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17469593053589950972
physical_device_desc: "device: 0, name: TITAN RTX, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [80]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, LSTM, Bidirectional
from tensorflow.keras.optimizers import RMSprop, SGD

In [81]:
import tensorflow as tf

with tf.device('/GPU:0'):

    model = Sequential()
    model.add(LSTM(800, input_shape=(time_step, 768), return_sequences=True))
    model.add(LSTM(10))
#     model.add(Dropout(0.5))
    # model.add(LSTM(64, return_sequences = True))
#     model.add(Dropout(0.4))
#     model.add(LSTM(150, return_sequences = True))
#     model.add(LSTM(32))
#     model.add(Dropout(0.5))
    # model.add(Activation('softmax'))
    # model.add(Dense(32, activation = 'tanh'))
    # model.add(Dropout(0.4))
#     model.add(Dense(128, activation = 'relu'))
#     model.add(Dense(64, activation = 'relu'))
#     model.add(Dense(32, activation = 'relu'))
    model.add(Dense(num_classes, activation = 'softmax'))

    sgd = SGD(lr=1e-1, decay=1e-6, momentum=0.9, nesterov=True)
    #rms = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

    model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [82]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_18 (LSTM)               (None, 1, 800)            5020800   
_________________________________________________________________
lstm_19 (LSTM)               (None, 10)                32440     
_________________________________________________________________
dense_24 (Dense)             (None, 7)                 77        
Total params: 5,053,317
Trainable params: 5,053,317
Non-trainable params: 0
_________________________________________________________________


In [83]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)


In [84]:
class_weights = {}

for i in range(7):
    class_weights[i] = arr[i]

In [85]:

with tf.device('/GPU:0'):

    hist = model.fit(X_train, y_train, 
#                  validation_data = (X_dev, y_dev), 
                 validation_split = 0.6,
                 verbose=1, 
                 epochs=100, batch_size=2, 
                 shuffle=False, callbacks=[early_stopping],
                 class_weight = class_weights)

Epoch 1/100
1998/1998 [==============================] - 38s 18ms/step - loss: 11.1169 - accuracy: 0.1588 - val_loss: 2.4945 - val_accuracy: 0.1188
Epoch 2/100
 451/1998 [=====>........................] - ETA: 17s - loss: 12.1679 - accuracy: 0.1517

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
# plt.subplot(1, 2, 1)
plt.plot(hist.history['loss'])
plt.title('Loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
def get_tfarray(arr):
    result = []
    for i in arr:
        m = np.max(i)
        result.append(i == m)
    return np.array(result)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
class_names = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']


y_pred = model.predict(X_test)

y_pred = get_tfarray(y_pred)

accuracy_score(y_test, y_pred, normalize=False)
print(classification_report(y_test, y_pred, target_names=class_names, digits=4))

In [ ]:
from sklearn.metrics import confusion_matrix
#Get the confusion matrix


#array([[1, 0, 0],
#   [1, 0, 0],
#   [0, 1, 2]])
cm =confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
#Now the normalize the diagonal entries
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#array([[1.        , 0.        , 0.        ],
#      [1.        , 0.        , 0.        ],
#      [0.        , 0.33333333, 0.66666667]])

#The diagonal entries are the accuracies of each class
cm.diagonal()

In [ ]:
predictions = model.predict(X_test)

In [ ]:
y_test[0], predictions[0]

In [ ]:
y_test[1], predictions[1]